# Common function shared

By default, **your databases** will be stored in Naas in the **"Inputs"** folder of each tool.<br>
You can edit it by changing the path of the outputs below if you want to connect to any kind of other tools.

*LinkedIn:*
- Profile posts stats
- Company posts stats
- Profile connections
- Company followers
- Profile posts engagements
- Company posts engagements

*YouTube:*
- Video stats

By default, all **your assets** will be store in Naas in the **"Outputs"**:
- Image
- HTML
- Post process data

## Input

### Import libraries

In [1]:
import naas
import pandas as pd
from naas_drivers import notion, linkedin, youtube
from datetime import datetime
import pytz
from os import path, environ, makedirs
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from naas_drivers.tools.notion import Link, BlockEmbed
import plotly.express as px

### Get data

In [2]:
def get_data(file_path, storage="Naas"):
    try:
        # Get CSV in Naas
        if storage == "Naas":
            if not naas.is_production():
                try:
                    input_path = path.join(PROD_DIR, file_path)
                    df = pd.read_csv(input_path) 
                except:
                    input_path = path.join(DEV_DIR, file_path)
                    df = pd.read_csv(input_path)
                    return df
            else:
                input_path = path.join(PROD_DIR, file_path)
                df = pd.read_csv(input_path) 
        # Get AWS 
        elif storage == "AWS":
            df = wr.s3.read_parquet(file_path, dataset=True)
    except Exception as e:
        print(e)
        # Empty dataframe returned
        return pd.DataFrame()
    return df

### Setup Variables

In [3]:
# Timezone
TIMEZONE = "Europe/Paris"

# Month Rolling
MONTH_ROLLING = 12

# MTD
PERIOD_MTD = "%Y-%m"
ORDER_MTD = "%Y%m"
DISPLAY_MTD = "%b %Y"
TEXT_MTD = "This month"
TEXT2_MTD = "months"
SCENARIO_MTD = "MTD"
CURRENT_MTD = datetime.now().strftime(PERIOD_MTD)

# Datetime format
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"

# Naas directory
PROJECT = naas.secret.get("GIT_REPO_NAME")
PROJECT_PATH = naas.secret.get("GIT_REPO_PATH")

# Manage DIR
PROD_DIR = path.join("/", "home", "ftp", "⚡ → Production", PROJECT)
DEV_DIR = PROJECT_PATH
if naas.is_production():
    DIR = PROD_DIR
else:
    DIR = DEV_DIR

### Setup LinkedIn

In [15]:
# LinkedIn cookies
LI_AT = naas.secret.get("LI_AT") # EXAMPLE : "AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2"
JSESSIONID = naas.secret.get("JSESSIONID") # EXAMPLE : "ajax:8379907400220387585"
LINKEDIN_PROFILE_URL = naas.secret.get("LINKEDIN_PROFILE_URL") # EXAMPLE : "ajax:8379907400220387585"
LINKEDIN_COMPANY_URL = naas.secret.get("LINKEDIN_COMPANY_URL") # EXAMPLE : "ajax:8379907400220387585"

# Outputs folders
LINKEDIN_INPUTS = path.join("LinkedIn", "Inputs")
if not path.exists(path.join(DIR, LINKEDIN_INPUTS)):
    makedirs(path.join(DIR, LINKEDIN_INPUTS))

LINKEDIN_OUTPUTS = path.join("LinkedIn", "Outputs")
if not path.exists(path.join(DIR, LINKEDIN_OUTPUTS)):
    makedirs(path.join(DIR, LINKEDIN_OUTPUTS))

# Get LinkedIn ID
def get_identity(file_path, identity):
    df = pd.DataFrame()
    file_path = path.join(DIR, file_path)
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError as e:
        # Empty dataframe returned
        if identity == "profile":
            if naas.secret.get("LINKEDIN_PROFILE_URL"):
                df = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(LINKEDIN_PROFILE_URL)
        elif identity == "company":
            if naas.secret.get("LINKEDIN_COMPANY_URL"):
                df = linkedin.connect(LI_AT, JSESSIONID).company.get_info(LINKEDIN_COMPANY_URL)
        if len(df) > 0:
            df.to_csv(file_path, index=False)
            naas.dependency.add(file_path)
            return df
    return df

# LinkedIn Profile info
LK_PROFILE_IDENTITY = path.join(LINKEDIN_INPUTS, "LINKEDIN_PROFILE.csv")
df_profile = get_identity(file_path=LK_PROFILE_IDENTITY, identity="profile")
LK_PROFILE_ID = ""
LK_FULLNAME = ""
if len(df_profile) > 0:
    LK_PROFILE_ID = df_profile.loc[0, "PROFILE_ID"]
    LK_FIRSTNAME = df_profile.loc[0, "FIRSTNAME"]
    LK_LASTNAME = df_profile.loc[0, "LASTNAME"]
    LK_FULLNAME = f"{LK_FIRSTNAME} {LK_LASTNAME}"

# LinkedIn Company info
LK_COMPANY_INFO = path.join(LINKEDIN_INPUTS, "LINKEDIN_COMPANY.csv")
df_company = get_identity(file_path=LK_COMPANY_INFO, identity="company")
LK_COMPANY_ID = ""
LK_COMPANY_NAME = ""
if len(df_company) > 0:
    LK_COMPANY_ID = df_company.loc[0, "COMPANY_ID"]
    LK_COMPANY_NAME = df_company.loc[0, "COMPANY_NAME"]

# Linkedin Stype
LINKEDIN = "LinkedIn"
LINKEDIN_LOGO = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/LinkedIn_logo_initials.png/800px-LinkedIn_logo_initials.png"
LINKEDIN_COLOR = "#1293d2"
LINKEDIN_COLOR2 = "#cfe9f6"

# LinkedIn Profile DATABASES
LK_PROFILE_POSTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_PROFILE_POSTS_{LK_PROFILE_ID}.csv')
LK_PROFILE_CONNECTIONS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_PROFILE_CONNECTIONS_{LK_PROFILE_ID}.csv')
LK_PROFILE_POSTS_LIKES = path.join(LINKEDIN_INPUTS, f'LINKEDIN_PROFILE_POSTS_LIKES_{LK_PROFILE_ID}.csv')
LK_PROFILE_POSTS_COMMENTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_PROFILE_POSTS_COMMENTS_{LK_PROFILE_ID}.csv')
LK_PROFILE_ENGAGEMENTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_PROFILE_ENGAGEMENTS_{LK_PROFILE_ID}.csv')

# LinkedIn Company DATABASES
LK_COMPANY_POSTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_COMPANY_POSTS_{LK_COMPANY_ID}.csv')
LK_COMPANY_FOLLOWERS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_COMPANY_FOLLOWERS_{LK_COMPANY_ID}.csv')
LK_COMPANY_POSTS_LIKES = path.join(LINKEDIN_INPUTS, f'LINKEDIN_COMPANY_POSTS_LIKES_{LK_COMPANY_ID}.csv')
LK_COMPANY_POSTS_COMMENTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_COMPANY_POSTS_COMMENTS_{LK_COMPANY_ID}.csv')
LK_COMPANY_ENGAGEMENTS = path.join(LINKEDIN_INPUTS, f'LINKEDIN_COMPANY_ENGAGEMENTS_{LK_COMPANY_ID}.csv')

### Setup YouTube

In [ ]:
# YouTube credentials
YOUTUBE_API_KEY = naas.secret.get('YOUTUBE_API_KEY')
YOUTUBE_CHANNEL_URL = naas.secret.get('YOUTUBE_CHANNEL_URL')

# Outputs folders
YOUTUBE_INPUTS = path.join("YouTube", "Inputs")
if not path.exists(path.join(DIR, YOUTUBE_INPUTS)):
    makedirs(path.join(DIR, YOUTUBE_INPUTS))

YOUTUBE_OUTPUTS = path.join("YouTube", "Outputs")
if not path.exists(path.join(DIR, YOUTUBE_OUTPUTS)):
    makedirs(path.join(DIR, YOUTUBE_OUTPUTS))

# YouTube Channel info
YT_CHANNEL_ID = YOUTUBE_CHANNEL_URL.split("/channel/")[-1].split("/")[0]

# YouTube
YOUTUBE = "YouTube"
YOUTUBE_LOGO = "https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/YouTube_social_white_square_%282017%29.svg/300px-YouTube_social_white_square_%282017%29.svg.png"
YOUTUBE_COLOR = "#FF0000"

# YouTube videos
YT_VIDEOS = path.join(YOUTUBE_INPUTS, f'YOUTUBE_VIDEOS_{YT_CHANNEL_ID}.csv')

### Setup Twitter

In [ ]:
# Outputs folders
TWITTER_INPUTS = path.join("Twitter", "Inputs")
if not path.exists(path.join(DIR, TWITTER_INPUTS)):
    makedirs(path.join(DIR, TWITTER_INPUTS))

TWITTER_OUTPUTS = path.join("Twitter", "Outputs")
if not path.exists(path.join(DIR, TWITTER_OUTPUTS)):
    makedirs(path.join(DIR, TWITTER_OUTPUTS))

# Twitter
TWITTER = "Twitter"
TWITTER_LOGO = "https://sapiens-uspc.com/wp-content/uploads/2017/10/twitter-logo-vector.png"
TWITTER_COLOR = "#1DA1F2"

### Setup Notion

In [ ]:
NOTION_CONTENT_DB = naas.secret.get("NOTION_CONTENT_DB")
NOTION_TOKEN = naas.secret.get("NOTION_TOKEN")

## Model

### Get notion content db

In [ ]:
def get_notion_df(database_url):
    # Get pages
    database_id = database_url.split("/")[-1].split("?v=")[0]
    pages = notion.connect(NOTION_TOKEN).database.query(database_id, query={})
    if len(pages) == 0:
        return pd.DataFrame()
    
    # Rename columns
    columns = pages[0].df().Name.tolist()
    to_rename = {}
    for i, v in enumerate(columns):
        to_rename[i] = v
    
    # Create dataframe from pages
    df = pd.DataFrame()
    for page in pages:
        values = page.df().drop("Type", axis=1).to_dict().get("Value")
        tmp_df = pd.DataFrame([values]).rename(columns=to_rename)
        content_url = tmp_df.loc[0, "Content URL"]
        if content_url == "None":
            notion.connect(NOTION_TOKEN).blocks.delete(page.id)
        else:
            df = pd.concat([df, tmp_df])

    df = df.sort_values(by="Publication Date", ascending=False).reset_index(drop=True)
    print("✅ Notion content DB:", len(df))
    return df

### Create barchart

In [ ]:
def create_barchart(df,
                    label="DATE_D",
                    value="VALUE",
                    value_d="VALUE_D",
                    text="TEXT"):
    # Init
    fig = go.Figure()
    
    # Create fig
    fig.add_trace(
        go.Bar(
            x=df[label],
            y=df[value],
            text=df[value_d],
            textposition="outside",
            hoverinfo="text",
            hovertext=df[text],
            marker=dict(color=COLOR)
        )
    )
    # Add logo
    fig.add_layout_image(
        dict(
            source=LOGO,
            xref="paper",
            yref="paper",
            x=0.01,
            y=1.045,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom"
        )
    )
    fig.update_traces(showlegend=False)
    # Plotly: Create title
    total_value = "{:,.0f}".format(df[value].sum()).replace(",", " ")
    varv = df.loc[df.index[-1], "VARV"]
    varp = df.loc[df.index[-1], "VARP"]
    varv_d = "{:,.0f}".format(varv).replace(",", " ")
    varp_d = "{:,.0%}".format(varp).replace(",", " ")
    if varv >= 0:
        varv_d = f"+{varv_d}"
        varp_d = f"+{varp_d}"
    title = f"<b><span style='font-size: 20px;'>{TITLE}</span></b><br><span style='font-size: 18px;'>Total: {total_value} | {PERIOD_TEXT}: {varv_d} ({varp_d})</span>"
    fig.update_layout(
        title=title,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        margin_pad=10,
    )
    fig.show()
    return fig

### Create barlinechart

In [ ]:
def create_barlinechart(df,
                        label="DATE_D",
                        value="VALUE",
                        value_d="VALUE_D",
                        text="TEXT",
                        yaxes_left=None,
                        yaxes_right=None):
    # Init
    fig = go.Figure()
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(
            x=df[label],
            y=df["VARV"],
            textposition="outside",
            hoverinfo="text",
            hovertext=df[text],
            marker=dict(color=COLOR2)
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            x=df[label],
            y=df[value],
            mode="lines+markers+text",
            hoverinfo="text",
            text=df[value_d],
            textposition="top left",
            hovertext=df[text],
            line=dict(color=COLOR, width=4),
        ),
        secondary_y=True,
    )
    # Set y-axes titles
    fig.update_yaxes(
        title_text=yaxes_left,
        title_font=dict(family="Arial", size=14, color="black"),
        secondary_y=False
    )
    fig.update_yaxes(
        title_text=yaxes_right,
        title_font=dict(family="Arial", size=14, color="black"),
        secondary_y=True
    )
    # Add logo
    fig.add_layout_image(
        dict(
            source=LOGO,
            xref="paper",
            yref="paper",
            x=0.01,
            y=1.045,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom"
        )
    )
    fig.update_traces(showlegend=False)
    # Plotly: Create title
    total_value = "{:,.0f}".format(df.loc[df.index[-1], "VALUE"]).replace(",", " ")
    varv = df.loc[df.index[-1], "VARV"]
    varp = df.loc[df.index[-1], "VARP"]
    varv_d = "{:,.0f}".format(varv).replace(",", " ")
    varp_d = "{:,.0%}".format(varp).replace(",", " ")
    if varv >= 0:
        varv_d = f"+{varv_d}"
        varp_d = f"+{varp_d}"
    title = f"<b><span style='font-size: 20px;'>{TITLE}</span></b><br><span style='font-size: 18px;'>Total: {total_value} | {PERIOD_TEXT}: {varv_d} ({varp_d})</span>"
    fig.update_layout(
        title=title,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        margin_pad=10,
    )
    fig.show()
    return fig

### Calc variation

In [ ]:
def calc_value(df_init, value="VALUE", value_comp="VALUE_COMP", varv="VARV", varp="VARP"):
    v1 = 0
    v2 = 0
    v3 = 0
    v4 = 0
    df = df_init.reset_index(drop=True)
    if len(df) > 0:
        if value in df.columns:
            v1 = df.loc[df.index[-1], value]
        if value_comp in df.columns:
            v2 = df.loc[df.index[-1], value_comp]
        if varv in df.columns:
            v3 = df.loc[df.index[-1], varv]
        if varp in df.columns:
            v4 = df.loc[df.index[-1], varp]
        return v1, v2, v3, v4
    else:
        return 0, 0, 0, 0

## Output

### Save data

In [ ]:
def save_data(df, file_path, storage="Naas"):
    output_path = None
    try:
        if storage == "Naas":
            if not naas.is_production():
                output_path = path.join(DEV_DIR, file_path)
                df.to_csv(output_path, index=False)
                naas.dependency.add(output_path)
            else:
                output_path = path.join(PROD_DIR, file_path)
                df.to_csv(output_path, index=False)
            print("✅ Dataframe successfully saved in CSV:", file_path)
        elif storage == "AWS":
            output_path = file_path
            wr.s3.to_parquet(
                df=df.astype(str),
                path=file_path,
                dataset=True,
                mode="overwrite"
            )
            print("✅ Dataframe successfully saved in AWS:", file_path)
    except Exception as e:
        return e
#     return output_path

### Save asset

In [ ]:
def save_graph_asset(obj, file_path, image=False):
    # Get output
    if naas.is_production():
        output_path = path.join(PROD_DIR, file_path)
    else:
        output_path = path.join(DEV_DIR, file_path)
        
    # Save asset
    if file_path.endswith(".csv"):
        obj.to_csv(output_path, index=False)
    elif file_path.endswith(".html"):
        obj.write_html(output_path)
    elif file_path.endswith(".png") and not image:
        obj.write_image(output_path)
    elif file_path.endswith(".png") and image:
        obj.to_file(output_path)
    return output_path

### Update "Report" Notion DB

In [ ]:
def update_report_status(report,
                         platform,
                         owner,
                         df=pd.DataFrame(),
                         image_link=None,
                         html_link=None,
                         csv_link=None,
                         page_icon=None,
                         ):
    # Manage page icon
    if page_icon is None:
        if platform == LINKEDIN:
            page_icon = LINKEDIN_LOGO
        elif platform == YOUTUBE:
            page_icon = YOUTUBE_LOGO
        elif platform == TWITTER:
            page_icon = TWITTER_LOGO
    # Decode database id
    database_id = naas.secret.get(name="NOTION_DATABASE_URL").split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id)
    
    # Create or update page
    page_new = True
    for page in pages:
        page_temp = page.df()
        page_id = page_temp.loc[page_temp.Name == "Report name", "Value"].values
        page_id2 = page_temp.loc[page_temp.Name == "Platform", "Value"].values
        page_id3 = page_temp.loc[page_temp.Name == "Owner", "Value"].values
        if page_id == report and page_id2 == platform and page_id3 == owner:
            page_new = False
            break
    try:
        if page_new:
            page = notion.connect(NOTION_TOKEN).Page.new(database_id=database_id).create()
            page.title("Report name", report)
            page.multi_select("Platform", [platform])
            page.select("Owner", owner)
            notion.client.pages.update(page_id=page.id, icon={'type': 'external', 'external': {'url': page_icon}})
            
        # Check if image already exists
        blocks = page.get_blocks()
        for block in blocks:
            content_block = getattr(block, block.type)
            if block.type == "image":
                image_url = block.image.external.url
                if image_url == image_link:
                    notion.connect(NOTION_TOKEN).blocks.delete(block.id)
            if block.type == "paragraph":
                if len(block.paragraph.text) > 0:
                    text = block.paragraph.text[0].text.content
                    if text == "Open dynamic chart":
                        notion.connect(NOTION_TOKEN).blocks.delete(block.id)
                    if text == "Download CSV":
                        notion.connect(NOTION_TOKEN).blocks.delete(block.id)
        if image_link:
            page.image(image_link)
        if html_link:
            res = page.paragraph("Open dynamic chart")
            res.paragraph.text[0].href = html_link
            res.paragraph.text[0].text.link = Link(html_link)
            res = page.rich_text("Chart", "Open dynamic chart")
            res.rich_text[0].href = html_link
            res.rich_text[0].text.link = Link(html_link)
        if csv_link:
            res = page.paragraph("Download CSV")
            res.paragraph.text[0].href = csv_link
            res.paragraph.text[0].text.link = Link(csv_link)
                
        # Update value
        value, value_comp, varv, varp = calc_value(df)
        if value != 0:
            page.number("This month", float(value))
        if value_comp != 0:
            page.number("Last month", float(value_comp))
        if varv != 0:
            page.number("Variation", float(varv))
        if varp != 0:
            page.number("Variation %", round(float(varp), 4))
        page.date("Last updated date", datetime.now(pytz.timezone(TIMEZONE)).strftime("%Y-%m-%d %H:%M:%S%z"))

        # Create page in Notion
        page.update()
        print(f"✅ Page '{report} - {platform} - {owner}' updated in Notion.")
    except Exception as e:
        raise(e)

In [ ]:
def update_dynamic_properties(page, row):
    # Page properties : dynamic
    page.number("Engagment score", round(float(row.ENGAGEMENT_SCORE), 4))
    page.number("Views", int(row.VIEWS))
    page.number("Likes", int(row.LIKES))
    page.number("Comments", int(row.COMMENTS))
    page.number("Shares", int(row.SHARES))
    return page

In [ ]:
def update_content_notion(df, database_url):
    # Decode database id
    database_id = database_url.split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id, query={})
    
    # Manage dataframe empty
    if len(df) == 0:
        print(f"🛑 Nothing to update in Notion.")
        return
    
    # Loop in data
    fillna = {
        "AUTHOR_URL": "",
        "TITLE": "",
        "TEXT": "",
        "COMPANY_MENTION": "",
        "PROFILE_MENTION": "",
        "TAGS": "",
        "TAGS_COUNT": 0,
        "LINKS": "",
        "LINKS_COUNT": 0,
        "EMOJIS": "",
        "EMOJIS_COUNT": 0,
        "CONTENT_TITLE": "",
        "CONTENT_URL": "",
        "CONTENT_ID": "",
        "POLL_ID": "",
        "POLL_QUESTION": "",
        "POLL_RESULTS": "",
        "IMAGE_URL": ""
    }
    df = df.fillna(fillna)
    for i, row in df.iterrows():
        title = row.TITLE
        content_title = row.CONTENT_TITLE
        if title is None and content_title is not None:
            title = f"Repost - {content_title}"
        elif title is None and content_title is None:
            title = "Repost"
        post_url = row.POST_URL
        print(post_url)
        
        # Create or update page
        page_new = True
        for page in pages:
            page_temp = page.df()
            page_id = page_temp.loc[page_temp.Name == "Content URL", "Value"].values
            if page_id == post_url:
                page_new = False
                break
        try:
            if page_new:
                # Create new page in notion
                page = notion.Page.new(database_id=database_id).create()

                # Page properties : static
                page.date("Publication Date", row.PUBLISHED_DATE)
                page.title("Name", title)
                page.select("Content type", row.CONTENT)
                page.select("Platform", "LinkedIn")
                page.select("Status", "Published ✨")
                page.select("Author", row.AUTHOR_NAME)
                profile_mention = row.PROFILE_MENTION
                if profile_mention is not None:
                    if len(profile_mention) > 2:
                        page.rich_text("Profile mention", profile_mention)
                company_mention = row.COMPANY_MENTION
                if company_mention is not None:
                    if len(company_mention) > 2:
                        page.rich_text("Company mention", company_mention)
                page.number("Nb tags", int(row.TAGS_COUNT))
                tags = row.TAGS
                if tags is None:
                    tags = ""
                else:
                    if len(tags) < 2:
                        tags = ""
                page.rich_text("Tags", tags)
                page.number("Nb emojis", int(row.EMOJIS_COUNT))
                emojis = row.EMOJIS
                if emojis is None:
                    emojis = ""
                else:
                    if len(emojis) < 2:
                        emojis = ""
                page.rich_text("Emojis", emojis)
                page.number("Nb links", int(row.LINKS_COUNT))
                links = row.LINKS
                if links is not None:
                    if len(links) > 2:
                        page.link("Links", links)
                page.number("Nb characters", int(row.CHARACTER_COUNT))
                page.link("Content URL", post_url)
                
                # Page blocks text
                page.heading_1("Text")
                text = row.TEXT
                if text is not None:
                    split_text = text.split("\n")
                    for t in split_text:
                        page.paragraph(t)
                    
                # Page blocks content
                image_url = row.IMAGE_URL
                content_url = row.CONTENT_URL
                poll_question = row.POLL_QUESTION
                if image_url or content_title or content_url or poll_question:
                    page.heading_1("Content")
                
                # Add image in content section
                if image_url:
                    page.heading_2("Image")
                    page.paragraph(image_url)
                    
                # Add post in content section
                if content_title:
                    page.heading_2("Media")
                    page.heading_3(content_title)
                
                if content_url:
                    page.paragraph(content_url)
                
                # Add poll graph in content section
                if poll_question:
                    page.heading_3("Poll")
                    page.paragraph(row.POLL_RESULTS)
                
                # Page properties : dynamic
                page = update_dynamic_properties(page, row)
                
                # Create page in Notion
                page.update()
                print(f"✅ Page '{title}' created in Notion.", '\n')
            else:
                # Page properties : dynamic
                page = update_dynamic_properties(page, row)
                
                # Update page
                page.update()
                print(f"📈 Post stats updated in notion for page '{title}'.", '\n')
        except Exception as e:
            print(f"❌ Error creating page '{title}' in Notion", e)
            print(row)
            raise(e)